In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
PATH = r"/Users/Vesal/Desktop/AltaML/Project/Use Case 1/Data/OneDrive_1_1-13-2021/"
FILE_NAME = 'Reduced_FHX_file.txt'

data = open(PATH + FILE_NAME, 'r').read().strip()

In [53]:
all_modules = data.split('\nMODULE_CLASS')
first_module = [all_modules[0]]
rest_of_modules = ['MODULE_CLASS' + module for module in all_modules[1:]]
modules = first_module + rest_of_modules

In [4]:
def module_class_variables(module):
    
    # name:
    name = module
    regex=re.compile(r'NAME=.* CATEGORY')
    module_name = regex.findall(name)[0][6:-10]

    # category
    category = module
    regex=re.compile(r'CATEGORY=\".*')
    module_category = regex.findall(category)[0][10:-1]
    
    return module_name, module_category

In [33]:
def module_instance_variables(module, module_name, feature):
    
    instance_modul_1 = module.split('MODULE_INSTANCE')[1:]
    
    module_instance_lists = []
    
    if len(instance_modul_1) > 0:
        instance_modul_1 = ['MODULE_INSTANCE' + inst_modu for inst_modu in  instance_modul_1]
        
        for inst in instance_modul_1:

            # tag:
            regex=re.compile(r'TAG=.* PLANT_AREA')
            tag = regex.findall(inst)[0][5:-12]

            # definition
            regex=re.compile(r'MODULE_CLASS=.* CATEGORY')
            module_class = regex.findall(inst)[0][14:-10]
            
            module_instance_lists.append([module_name, feature, module_class, tag])


        return module_instance_lists
    
    else:
        return [[module_name, feature, np.nan, np.nan]]

In [34]:
def function_block_variables(module, module_name, feature):

    func_modul_1 = module.split('FUNCTION_BLOCK')[1:]
    
    func_lists = []
    
    if len(func_modul_1) > 0:
    
        func_modul_1 = ['FUNCTION_BLOCK' + func for func in  func_modul_1]
        
        for func in func_modul_1:

            # name:
            regex=re.compile(r'NAME=.* DEFINITION')
            func_name = regex.findall(func)[0][6:-12]

            # definition
            regex=re.compile(r'DEFINITION=.*')
            func_def = regex.findall(func)[0][12:-1]
            
            func_lists.append([module_name, feature, func_name, func_def])

        return func_lists
    
    else:
        return [[module_name, feature, np.nan, np.nan]]

In [35]:
def attribute_variables(module, module_name, feature):
    
    attr_modul_1 = module.split('ATTRIBUTE NAME')[1:]
    
    if len(attr_modul_1) > 0:
        
        attr_modul_1 = ['ATTRIBUTE NAME' + attr for attr in  attr_modul_1]
        
        attribute_lists = []
        
        for atr in attr_modul_1:
        
            # attribute name
            regex=re.compile(r'NAME=.* TYPE')
            attr_name = regex.findall(atr)[0][6:-6]

            # attribute type
            regex=re.compile(r'TYPE=.*')
            attr_type = regex.findall(atr)[0][5:]
            
            attribute_lists.append([module_name, feature, attr_name, attr_type])
            
        return attribute_lists
    
    else:
        return [[module_name, feature, np.nan, np.nan]]

In [36]:
def wire_connections_variables(module, module_name, feature):

    wire_modul_1 = module.split('WIRE SOURCE')[1:]
    
    if len(wire_modul_1) > 0:
        wire_modul_1 = ['WIRE SOURCE' + wire for wire in  wire_modul_1]
        
        wire_connection_lists = []
        
        for conn in wire_modul_1:

            # source
            regex=re.compile(r'SOURCE=.* D')
            source = regex.findall(conn)[0][8:-3]

            # destination
            regex=re.compile(r'DESTINATION=.*')
            destination = regex.findall(conn)[0][13:-1]
            
            wire_connection_lists.append([module_name, feature, source, destination])

        return wire_connection_lists
    else:
        return [[module_name, feature, np.nan, np.nan]]

In [37]:
def attribute_instance_variables(module, module_name, feature):

    attr_inst_modul_1 = module.split('ATTRIBUTE_INSTANCE')[1:]
    
    if len(attr_inst_modul_1) > 0:
        attr_inst_modul_1 = ['ATTRIBUTE_INSTANCE' + attr for attr in  attr_inst_modul_1]
        
        attr_instance_lists = []
        
        for attr_inst in attr_inst_modul_1:
            
            # attribute instant name
            regex=re.compile(r'NAME=.*')
            attr_instance_name = regex.findall(attr_inst)[0][6:-1]
            
            attr_instance_lists.append([module_name, feature, attr_instance_name, np.nan])

        return attr_instance_lists

    else:
        print('im in else')
        return [[module_name, feature, np.nan, np.nan]]

In [38]:
features = ['function_block', 'attribute', 'wire_connections', 'attribute_instance', 'module_instance']
# features = ['attribute_instance']

In [39]:
module_list = []
for mod in modules:
    module_name , module_category = module_class_variables(mod)
    for feature in features:
        exec(f'var1= {feature}_variables(mod, module_name, feature)')
        module_list.append(var1)

In [55]:
flat_list = [item for sublist in module_list for item in sublist]
# flat_list

In [41]:
final_df = pd.DataFrame(flat_list, columns=['module_name', 'feature', 'var1', 'var2'])
final_df

,module_name,feature,var1,var2
0,KS_AI_STD,function_block,PV-COMPARE,CALC
1,KS_AI_STD,function_block,HI_HI_ALARM,CND
2,KS_AI_STD,function_block,HI_ALARM,CND
3,KS_AI_STD,function_block,LO_ALARM,CND
4,KS_AI_STD,function_block,LO_LO_ALARM,CND
...,...,...,...,...
1005,KS_AI_STD,attribute_instance,LO_LO_ALARM/T_EXPRESSION,NaN
1006,KS_AI_STD,attribute_instance,LO_LO_ALARM/TIME_DURATION,NaN
1007,KS_AI_STD,attribute_instance,LO_LO_ALARM/ERROR_OPT,NaN
1008,KS_AI_STD,attribute_instance,PV-COMPARE/T_EXPRESSION,NaN


In [42]:
final_df[final_df['module_name'] == 'KS_AI_STD']

,module_name,feature,var1,var2
0,KS_AI_STD,function_block,PV-COMPARE,CALC
1,KS_AI_STD,function_block,HI_HI_ALARM,CND
2,KS_AI_STD,function_block,HI_ALARM,CND
3,KS_AI_STD,function_block,LO_ALARM,CND
4,KS_AI_STD,function_block,LO_LO_ALARM,CND
...,...,...,...,...
1005,KS_AI_STD,attribute_instance,LO_LO_ALARM/T_EXPRESSION,NaN
1006,KS_AI_STD,attribute_instance,LO_LO_ALARM/TIME_DURATION,NaN
1007,KS_AI_STD,attribute_instance,LO_LO_ALARM/ERROR_OPT,NaN
1008,KS_AI_STD,attribute_instance,PV-COMPARE/T_EXPRESSION,NaN


In [43]:
wire_connections = final_df[final_df['feature'] == 'wire_connections'].rename(columns={'var1':'source',
                                                                                      'var2':'destination'})
wire_connections

,module_name,feature,source,destination
27,KS_AI_STD,wire_connections,ALM_HYS,HI_HI_ALM_HYS
28,KS_AI_STD,wire_connections,ALM_HYS,HI_ALM_HYS
29,KS_AI_STD,wire_connections,ALM_HYS,LO_ALM_HYS
30,KS_AI_STD,wire_connections,ALM_HYS,LO_LO_ALM_HYS
31,KS_AI_STD,wire_connections,HI_HI_ALARM/OUT_D,AI1/HI_HI_ENAB
...,...,...,...,...
933,KS_AI_STD,wire_connections,ALM_HYS,LO_LO_ALM_HYS
934,KS_AI_STD,wire_connections,HI_HI_ALARM/OUT_D,AI1/HI_HI_ENAB
935,KS_AI_STD,wire_connections,HI_ALARM/OUT_D,AI1/HI_ENAB
936,KS_AI_STD,wire_connections,LO_ALARM/OUT_D,AI1/LO_ENAB


In [44]:
function_blocks = final_df[final_df['feature'] == 'function_block'].rename(columns={'var1':'name',
                                                                                    'var2':'definition'})
function_blocks

,module_name,feature,name,definition
0,KS_AI_STD,function_block,PV-COMPARE,CALC
1,KS_AI_STD,function_block,HI_HI_ALARM,CND
2,KS_AI_STD,function_block,HI_ALARM,CND
3,KS_AI_STD,function_block,LO_ALARM,CND
4,KS_AI_STD,function_block,LO_LO_ALARM,CND
...,...,...,...,...
904,KS_AI_STD,function_block,HI_HI_ALARM,CND
905,KS_AI_STD,function_block,HI_ALARM,CND
906,KS_AI_STD,function_block,LO_ALARM,CND
907,KS_AI_STD,function_block,LO_LO_ALARM,CND


In [45]:
module_instance = final_df[final_df['feature'] == 'module_instance'].rename(columns={'var1':'class',
                                                                                    'var2':'tag'})
module_instance

,module_name,feature,class,tag
106,KS_AI_STD,module_instance,NaN,NaN
901,KS_AI_SIS_HH,module_instance,KS_AI_STD,TI_580920
902,KS_AI_SIS_HH,module_instance,KS_AI_SIS_HH,PI_580911B
1009,KS_AI_STD,module_instance,NaN,NaN


In [48]:
attribute = final_df[final_df['feature'] == 'attribute'].rename(columns={'var1':'name', 'var2':'type'})
attribute

,module_name,feature,name,type
6,KS_AI_STD,attribute,VERSION_CLASS,UINT32
7,KS_AI_STD,attribute,HI_HI_ALM,EVENT
8,KS_AI_STD,attribute,HI_ALM,EVENT
9,KS_AI_STD,attribute,LO_ALM,EVENT
10,KS_AI_STD,attribute,LO_LO_ALM,EVENT
...,...,...,...,...
925,KS_AI_STD,attribute,HI_ALM_HYS,INTERNAL_REFERENCE
926,KS_AI_STD,attribute,HI_HI_ALM_HYS,INTERNAL_REFERENCE
927,KS_AI_STD,attribute,LO_ALM_HYS,INTERNAL_REFERENCE
928,KS_AI_STD,attribute,LO_LO_ALM_HYS,INTERNAL_REFERENCE


In [49]:
attribute_instance = final_df[final_df['feature'] == 'attribute_instance'].rename(columns={'var1':'name'})
attribute_instance

,module_name,feature,name,var2
35,KS_AI_STD,attribute_instance,MCOMMAND,NaN
36,KS_AI_STD,attribute_instance,MSTATE,NaN
37,KS_AI_STD,attribute_instance,MERROR,NaN
38,KS_AI_STD,attribute_instance,MSTATUS,NaN
39,KS_AI_STD,attribute_instance,MSTATUS_MASK,NaN
...,...,...,...,...
1004,KS_AI_STD,attribute_instance,LO_LO_ALARM/DESC,NaN
1005,KS_AI_STD,attribute_instance,LO_LO_ALARM/T_EXPRESSION,NaN
1006,KS_AI_STD,attribute_instance,LO_LO_ALARM/TIME_DURATION,NaN
1007,KS_AI_STD,attribute_instance,LO_LO_ALARM/ERROR_OPT,NaN
